In [201]:
%pip install tensorflow==2.15 pillow

Note: you may need to restart the kernel to use updated packages.


## Путь, где хранится датасет

In [202]:
import pathlib

dataset_dir = pathlib.Path('./dataset-32x32rgb')

## Параметры картинок в датасете

### Высота и ширина картинки (разрешение)

In [203]:
img_height = 32
img_width = 32

### Количество цветовых каналов

In [204]:
color_channels = 3

## Параметры НС

### Сколько проб использовать для обновления весов за раз

In [205]:
batch_size = 32

### Число элементов в первом скрытом слое
Количество элементов в слое имеет зависимость `Ns*2^(n-1)`, где n - номер скрытого слоя

In [206]:
Ns = 16

### Kernel size (на какие кусочки Conv2D разбивает входные данные)
`kernel_size = 3` означает разбиение на кусочки 3x3 px

In [207]:
kernel_size = 3

### Кол-во итераций обучения

In [208]:
epochs = 10

## Загрузка датасета в train dataset и validation dataset
Отношение `4:1`

In [209]:
import keras

image_count = len(list(dataset_dir.glob('*/*.png')))

train_ds = keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    color_mode="rgb",
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    color_mode="rgb",
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 57627 files belonging to 66 classes.
Using 46102 files for training.
Found 57627 files belonging to 66 classes.
Using 11525 files for validation.


## Замена названий классов (букв) на human readable (а, б, в, ...)

In [210]:
import numpy as np

char = ord('а')
char_upper = ord('А')
yo = False
Yo = False

print("До: " + str(train_ds.class_names))

for i, v in enumerate(train_ds.class_names):
    if train_ds.class_names[i].startswith('00'):
       if (char == ord("ж") and not yo):
           train_ds.class_names[i] = 'ё'
           yo = True
           continue
       train_ds.class_names[i] = chr(char)
       char += 1
    else:
       if (char_upper == ord("Ж") and not Yo):
           train_ds.class_names[i] = 'Ё'
           Yo = True
           continue
       train_ds.class_names[i] = chr(char_upper)
       char_upper += 1

class_names = train_ds.class_names

print("После: " + str(class_names))


До: ['00_00_00', '00_01_00', '00_02_00', '00_03_00', '00_04_00', '00_05_00', '00_06_00', '00_07_00', '00_08_00', '00_09_00', '00_10_00', '00_11_00', '00_12_00', '00_13_00', '00_14_00', '00_15_00', '00_16_00', '00_17_00', '00_18_00', '00_19_00', '00_20_00', '00_21_00', '00_22_00', '00_23_00', '00_24_00', '00_25_00', '00_26_00', '00_27_00', '00_28_00', '00_29_00', '00_30_00', '00_31_00', '00_32_00', '01_00_00', '01_01_00', '01_02_00', '01_03_00', '01_04_00', '01_05_00', '01_06_00', '01_07_00', '01_08_00', '01_09_00', '01_10_00', '01_11_00', '01_12_00', '01_13_00', '01_14_00', '01_15_00', '01_16_00', '01_17_00', '01_18_00', '01_19_00', '01_20_00', '01_21_00', '01_22_00', '01_23_00', '01_24_00', '01_25_00', '01_26_00', '01_27_00', '01_28_00', '01_29_00', '01_30_00', '01_31_00', '01_32_00']
После: ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 

Сохраним названия классов для pretty предикта

In [211]:
np.savetxt('class_names-dataset.txt', class_names, fmt='%s')

### Определение слоев нейронной сети и компиляция модели

In [212]:
import tensorflow as tf
import keras.layers as layers

num_classes = len(class_names)

model = keras.models.Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, color_channels)),
  layers.Conv2D(Ns, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(Ns*2, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(Ns*2**2, kernel_size, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.1),
  layers.Flatten(),
  layers.Dense(Ns*2**3, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Характеристики слоев модели

In [213]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 32, 32, 3)         0         
                                                                 
 conv2d_57 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 max_pooling2d_57 (MaxPooli  (None, 16, 16, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_58 (Conv2D)          (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_58 (MaxPooli  (None, 8, 8, 32)          0         
 ng2D)                                                           
                                                                 
 conv2d_59 (Conv2D)          (None, 8, 8, 64)        

### Обучение модели и сохранение

In [214]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

model.save('russian-cursive-32x32rgb.model.keras')

model.summary()

Epoch 1/10


1441/1441 [==============================] - 14s 9ms/step - loss: 2.6969 - accuracy: 0.3342 - val_loss: 1.1842 - val_accuracy: 0.6765
Epoch 2/10
1441/1441 [==============================] - 14s 10ms/step - loss: 0.9378 - accuracy: 0.7362 - val_loss: 0.7358 - val_accuracy: 0.7918
Epoch 3/10
1441/1441 [==============================] - 15s 10ms/step - loss: 0.6374 - accuracy: 0.8147 - val_loss: 0.5526 - val_accuracy: 0.8393
Epoch 4/10
1441/1441 [==============================] - 15s 10ms/step - loss: 0.4963 - accuracy: 0.8536 - val_loss: 0.4316 - val_accuracy: 0.8723
Epoch 5/10
1441/1441 [==============================] - 15s 10ms/step - loss: 0.4125 - accuracy: 0.8757 - val_loss: 0.4035 - val_accuracy: 0.8818
Epoch 6/10
1441/1441 [==============================] - 14s 10ms/step - loss: 0.3479 - accuracy: 0.8937 - val_loss: 0.3343 - val_accuracy: 0.9002
Epoch 7/10
1441/1441 [==============================] - 14s 10ms/step - loss: 0.3039 - accuracy: 0.9061 - val_loss: 0.3527 - val_accurac